In [1]:
%pylab inline
from scipy.optimize import curve_fit
import pandas as pd
import pandas as pd
import seaborn as sns
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import matplotlib.style as style
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
import sys
import bookie_package as bp

import warnings
warnings.filterwarnings('ignore')


Populating the interactive namespace from numpy and matplotlib


In [2]:
df_both_seasons_home = pd.read_pickle('df_both_seasons_essentials')

## Add avg Home Team Goal Difference

In [3]:
d_both_seasons = bp.averages.avg_goal_diff(df_both_seasons_home, 'AVGHTGDIFF', 'HomeTeam', 'H')

In [4]:
df_both_seasons = bp.averages.from_dict_value_to_df(d_both_seasons)

In [5]:
df_both_seasons=df_both_seasons.sort_values(['Year', 'Month','Day'], ascending=False)

In [6]:
import pickle
df_both_seasons.to_pickle('df_both_seasons')

In [7]:
avg_fthg_per_team=bp.averages.avg_goals(df_both_seasons, 'AVGFTHG', 'HomeTeam', 'H')

In [8]:
df_both_seasons = bp.averages.from_dict_value_to_df(avg_fthg_per_team)

In [9]:
df_both_seasons=df_both_seasons.sort_values(['Year', 'Month','Day'], ascending=False)

In [10]:
df_both_seasons.head()

,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,...,HF,AF,HY,AY,HR,AR,HTGDIFF,ATGDIFF,AVGHTGDIFF,AVGFTHG
139,18,12,2019,Leverkusen,Hertha,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,1.760000
140,18,12,2019,M'gladbach,Paderborn,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.760000,2.080000
141,18,12,2019,Wolfsburg,Schalke 04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520000,1.800000
142,18,12,2019,Ein Frankfurt,FC Koln,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.720000,1.960000
143,18,12,2019,Freiburg,Bayern Munich,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.416667,1.791667


## Add Columns with previous HTGDIFF for each HomeTeam

In [11]:
team_with_past_HTGDIFF=bp.averages.previous_data(df_both_seasons, 'HomeTeam', 'HTGDIFF')

In [12]:
df_team_with_past_HTGDIFF = bp.averages.from_dict_value_to_df(team_with_past_HTGDIFF)

In [13]:
df_team_with_past_HTGDIFF = df_team_with_past_HTGDIFF.reindex(columns=[
    'Day', 'Month', 'Year', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG',
    'HTGDIFF', 'ATGDIFF', 'AVGHTGDIFF','AVGFTHG', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'HTGDIFF_1', 'HTGDIFF_2', 'HTGDIFF_3', 'HTGDIFF_4', 'HTGDIFF_5', 'HTGDIFF_6', 'HTGDIFF_7',
    'HTGDIFF_8', 'HTGDIFF_9', 'HTGDIFF_10', 'HTGDIFF_11', 'HTGDIFF_12', 'HTGDIFF_13', 'HTGDIFF_14', 'HTGDIFF_15', 'HTGDIFF_16', 'HTGDIFF_17', 'HTGDIFF_18', 'HTGDIFF_19',
    'HTGDIFF_20', 'HTGDIFF_21', 'HTGDIFF_22', 'HTGDIFF_23'
])

In [14]:
df_team_with_past_HTGDIFF.fillna(0, inplace=True)

In [15]:
team_with_past_HST=bp.averages.previous_data(df_team_with_past_HTGDIFF, 'HomeTeam', 'HST')

In [16]:
df_team_with_past_HST = bp.averages.from_dict_value_to_df(team_with_past_HST)

In [17]:

df_team_with_past_HST = df_team_with_past_HST.reindex(columns=[ 'Day',
    'Month',
    'Year',
    'HomeTeam',
    'AwayTeam',
    'FTHG',
    'FTAG',
    'HTGDIFF',
    'ATGDIFF',
    'AVGHTGDIFF',
    'AVGFTHG',
    'HS',
    'AS',
    'HST',
    'AST',
    'HC',
    'AC',
    'HF',
    'AF',
    'HY',
    'AY',
    'HR',
    'AR',
    'HTGDIFF_1',
    'HTGDIFF_2',
    'HTGDIFF_3',
    'HTGDIFF_4',
    'HTGDIFF_5',
    'HTGDIFF_6',
    'HTGDIFF_7',
    'HTGDIFF_8',
    'HTGDIFF_9',
    'HTGDIFF_10',
    'HTGDIFF_11',
    'HTGDIFF_12',
    'HTGDIFF_13',
    'HTGDIFF_14',
    'HTGDIFF_15',
    'HTGDIFF_16',
    'HTGDIFF_17',
    'HTGDIFF_18',
    'HTGDIFF_19',
    'HTGDIFF_20',
    'HTGDIFF_21',
    'HTGDIFF_22',
    'HTGDIFF_23',
    'HST_1',
    'HST_2',
    'HST_3',
    'HST_4',
    'HST_5',
    'HST_6',
    'HST_7',
    'HST_8',
    'HST_9',
    'HST_10',
    'HST_11',
    'HST_12',
    'HST_13',
    'HST_14',
    'HST_15',
    'HST_16',
    'HST_17',
    'HST_18',
    'HST_19',
    'HST_20',
    'HST_21',
    'HST_22',
    'HST_23' ])

In [18]:
team_with_past_FTHG = bp.averages.previous_data(df_team_with_past_HST, 'HomeTeam', 'FTHG')

In [19]:
df_team_with_past_FTHG = bp.averages.from_dict_value_to_df(team_with_past_FTHG)

In [20]:
  df_team_with_past_FTHG = df_team_with_past_FTHG.reindex(columns=[ 'Day',
    'Month',
    'Year',
    'HomeTeam',
    'AwayTeam',
    'FTHG',
    'FTAG',
    'HTGDIFF',
    'ATGDIFF',
    'AVGHTGDIFF',
    'AVGFTHG',
    'HS',
    'AS',
    'HST',
    'AST',
    'HC',
    'AC',
    'HF',
    'AF',
    'HY',
    'AY',
    'HR',
    'AR',
    'HTGDIFF_1',
    'HTGDIFF_2',
    'HTGDIFF_3',
    'HTGDIFF_4',
    'HTGDIFF_5',
    'HTGDIFF_6',
    'HTGDIFF_7',
    'HTGDIFF_8',
    'HTGDIFF_9',
    'HTGDIFF_10',
    'HTGDIFF_11',
    'HTGDIFF_12',
    'HTGDIFF_13',
    'HTGDIFF_14',
    'HTGDIFF_15',
    'HTGDIFF_16',
    'HTGDIFF_17',
    'HTGDIFF_18',
    'HTGDIFF_19',
    'HTGDIFF_20',
    'HTGDIFF_21',
    'HTGDIFF_22',
    'HTGDIFF_23',
    'HST_1',
    'HST_2',
    'HST_3',
    'HST_4',
    'HST_5',
    'HST_6',
    'HST_7',
    'HST_8',
    'HST_9',
    'HST_10',
    'HST_11',
    'HST_12',
    'HST_13',
    'HST_14',
    'HST_15',
    'HST_16',
    'HST_17',
    'HST_18',
    'HST_19',
    'HST_20',
    'HST_21',
    'HST_22',
    'HST_23',
    'FTHG_1',
    'FTHG_2',
    'FTHG_3',
    'FTHG_4',
    'FTHG_5',
    'FTHG_6',
    'FTHG_7',
    'FTHG_8',
    'FTHG_9',
    'FTHG_10',
    'FTHG_11',
    'FTHG_12',
    'FTHG_13',
    'FTHG_14',
    'FTHG_15',
    'FTHG_16',
    'FTHG_17',
    'FTHG_18',
    'FTHG_19',
    'FTHG_20',
    'FTHG_21',
    'FTHG_22',
    'FTHG_23' ])

In [21]:
df_team_with_past_FTHG.sort_values(['Year', 'Month','Day'], ascending=False,inplace=True)

In [22]:
df_team_with_past_FTHG.fillna(0, inplace=True)

In [23]:
df_team_with_past_FTHG = df_team_with_past_FTHG[['Day', 'Month', 'Year', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG',
    'HTGDIFF', 'ATGDIFF', 'AVGHTGDIFF', 'AVGFTHG','HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'HTGDIFF_1', 'HTGDIFF_2', 'HTGDIFF_3', 'HTGDIFF_4', 'HTGDIFF_5', 'HTGDIFF_6', 'HTGDIFF_7',
    'HTGDIFF_8', 'HTGDIFF_9', 'HTGDIFF_10',  'HST_1',
 'HST_2',
 'HST_3',
 'HST_4',
 'HST_5',
 'HST_6',
 'HST_7',
 'HST_8',
 'HST_9',
 'HST_10',
 'FTHG_1',
 'FTHG_2',
 'FTHG_3',
 'FTHG_4',
 'FTHG_5',
 'FTHG_6',
 'FTHG_7',
 'FTHG_8',
 'FTHG_9',
 'FTHG_10'
    ]]

In [24]:
df_team_with_past_FTHG.to_excel('df_team_with_past_FTHG_home.xlsx')

In [25]:
df_team_with_past_FTHG.columns

Index(['Day', 'Month', 'Year', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG',
       'HTGDIFF', 'ATGDIFF', 'AVGHTGDIFF', 'AVGFTHG', 'HS', 'AS', 'HST', 'AST',
       'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'HTGDIFF_1',
       'HTGDIFF_2', 'HTGDIFF_3', 'HTGDIFF_4', 'HTGDIFF_5', 'HTGDIFF_6',
       'HTGDIFF_7', 'HTGDIFF_8', 'HTGDIFF_9', 'HTGDIFF_10', 'HST_1', 'HST_2',
       'HST_3', 'HST_4', 'HST_5', 'HST_6', 'HST_7', 'HST_8', 'HST_9', 'HST_10',
       'FTHG_1', 'FTHG_2', 'FTHG_3', 'FTHG_4', 'FTHG_5', 'FTHG_6', 'FTHG_7',
       'FTHG_8', 'FTHG_9', 'FTHG_10'],
      dtype='object')

In [26]:
df_result = df_team_with_past_FTHG.copy()

In [27]:
df_result = df_result.drop(['HomeTeam', 'AwayTeam'], axis = 1)

In [28]:
print('Shape of features:', df_result.shape)

Shape of features: (450, 51)


## Features and Labels and Convert Data to Arrays

In [29]:
# Labels are the values we want to predict
labels = np.array(df_result['FTHG'])

# Remove the labels from the features
# axis 1 refers to the columns
df_result= df_result.drop(['Year','Day','Month','HTGDIFF_10','FTHG_6','FTHG_8', 'FTHG_9', 'FTHG_10','FTHG', 'FTAG', 'HTGDIFF', 'ATGDIFF', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR'], axis = 1)

# Saving feature names for later use
feature_list = list(df_result.columns)

# Convert to numpy array
df_result = np.array(df_result)

## Training and Testing Sets

In [30]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(
    df_result, labels, test_size = 0.25,random_state = 42
)
                

In [31]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (337, 27)
Training Labels Shape: (337,)
Testing Features Shape: (113, 27)
Testing Labels Shape: (113,)


## Establish Baseline

In [32]:
# The baseline predictions are the historical averages
baseline_preds = train_features[:, feature_list.index('AVGFTHG')]
# AVerage goals made by home team

# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - train_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2), 'Goals.')

Average baseline error:  1.1 Goals.


## Train Model

In [33]:
rf = bp.prediction.random_forrest(train_features, train_labels, n_estimators=1000,random_state = 42)

## Make Predictions on Test Data

In [34]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'Goals.')

Mean Absolute Error: 1.22 Goals.


In [35]:
#for i in zip(np.round(predictions,0), test_labels):
 #   print(i)

## Determine Performance Metrics

In [36]:
# Calculate mean absolute percentage error (MAPE)
err = errors/test_labels
err[np.isnan(err)] = 0
mape = err * 100
mape[mape == inf] = 0
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 61.96 %.


### Visualizing a Single Decision Tree

In [37]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree_home.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree_home.dot')
# Write graph to a png file
graph.write_png('tree_home.png')

In [38]:
print('The depth of this tree is:', tree.tree_.max_depth)

The depth of this tree is: 16


In [39]:
# Limit depth of tree to 2 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 4, random_state=42)
rf_small.fit(train_features, train_labels)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree_home.dot', feature_names = feature_list, rounded = True, precision = 1)

(graph, ) = pydot.graph_from_dot_file('small_tree_home.dot')

graph.write_png('small_tree_home.png');

### Variable Importances

In [40]:
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: AVGFTHG              Importance: 0.16517
Variable: AVGHTGDIFF           Importance: 0.10169
Variable: HST_1                Importance: 0.04986
Variable: HTGDIFF_7            Importance: 0.04686
Variable: HTGDIFF_8            Importance: 0.04234
Variable: HST_5                Importance: 0.03912
Variable: HTGDIFF_3            Importance: 0.03631
Variable: HST_2                Importance: 0.03557
Variable: HST_3                Importance: 0.03497
Variable: HST_10               Importance: 0.03411
Variable: HST_7                Importance: 0.03372
Variable: HST_4                Importance: 0.03351
Variable: HTGDIFF_1            Importance: 0.03308
Variable: HTGDIFF_2            Importance: 0.03287
Variable: HST_8                Importance: 0.02584
Variable: HTGDIFF_9            Importance: 0.0257
Variable: HTGDIFF_6            Importance: 0.02455
Variable: HST_6                Importance: 0.02376
Variable: HTGDIFF_4            Importance: 0.02351
Variable: FTHG_1               I

## Random Forest Optimization through Random Search

In [41]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
# Hyperparameter grid
param_grid = {
   'n_estimators': np.linspace(10, 1000).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
estimator = RandomForestClassifier(random_state = 42)

# Create the random search model
rs = RandomizedSearchCV(estimator, param_grid, n_jobs = -1, cv = 3, 
                        n_iter = 10, verbose = 1, random_state=42)

# Fit 
rs.fit(train_features,train_labels)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    9.0s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [42]:
best_params = rs.best_params_

In [43]:
rf = bp.prediction.random_forrest(
    train_features, train_labels, 
    n_estimators=best_params['n_estimators'],
    random_state = 42,
    min_samples_split = best_params['min_samples_split'],
    max_leaf_nodes = best_params['max_leaf_nodes'],
    max_features = best_params['max_features'],
    max_depth = best_params['max_depth'],
    bootstrap = best_params['bootstrap']
)


In [44]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'Goals.')

Mean Absolute Error: 1.21 Goals.


In [45]:
# Calculate mean absolute percentage error (MAPE)
err = errors/test_labels
err[np.isnan(err)] = 0
mape = err * 100
mape[mape == inf] = 0
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 63.06 %.


In [46]:
next_games=df_result[:9]
predictions_next_games = rf.predict(next_games)
next_games_predictions=np.round(predictions_next_games,0)

In [47]:
df_both_seasons.head(9)['FTHG'] = next_games_predictions
df_both_seasons.head(9)

,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,...,HF,AF,HY,AY,HR,AR,HTGDIFF,ATGDIFF,AVGHTGDIFF,AVGFTHG
139,18,12,2019,Leverkusen,Hertha,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,1.760000
140,18,12,2019,M'gladbach,Paderborn,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.760000,2.080000
141,18,12,2019,Wolfsburg,Schalke 04,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520000,1.800000
142,18,12,2019,Ein Frankfurt,FC Koln,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.720000,1.960000
143,18,12,2019,Freiburg,Bayern Munich,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.416667,1.791667
135,17,12,2019,Werder Bremen,Mainz,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125000,1.791667
136,17,12,2019,Dortmund,RB Leipzig,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.833333,3.125000
137,17,12,2019,Augsburg,Fortuna Dusseldorf,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125000,1.958333
138,17,12,2019,Union Berlin,Hoffenheim,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.375000,1.500000
